In [8]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Updated with my username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)

# Below added to fix Data Manipulation / Model
HOST = 'nv-desktop-services.apporto.com'
PORT = 33384  # My port 33384? Given port: 31580
DB = 'AAC'
COL = 'animals'

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))
if "_id" in df.columns:
    df.drop(columns=['_id'], inplace=True)

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('ProjectTwo') # change your project name

#Add your logo below
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.H3("Project Two - Elizabeth Marticello")),
    html.Center(html.A
                (html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height': '140px'}
                ),
                 href='https://www.snhu.edu', target='_blank') # change this webpage to link to your shelter
               ),
    html.Hr(),
    # This creates radio buttons to interactice with to filter the data table
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 
             'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 
             'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 
             'value': 'disaster'},
            {'label': 'Reset (All)', 
             'value': 'reset'}
        ],
        value='reset',
        inline=True
    ),
    # Code for the interactive filtering options. 
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # Features for the interactice data table to make it user-friendly for the client
        row_selectable="single",
        selected_rows=[0] if len(df) > 0 else [],
        page_size=10,
        sort_action="native",
        filter_action="native",
        style_table={'overflowX':'auto'},
        style_cell={'textAlign':'left', 'minWidth':'100px', 'whiteSpace':'normal'}
    ),
    html.Br(),
    html.Hr(),
    
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])


#############################################
# Interaction Between Components / Controller
#############################################

# Extra function to help select more breeds that are include 'mix' in breed
# I thought the filter wasn't working properly, this helped solve it. 
#def breeds_regex(breeds):
#    return{"$or": [
#        {"breed": {"$regex": rf"^{b}.*$", "$options": "i"}}
#        for b in breeds
#    ]}

# Filtering Information
def build_query(filter_type: str):
    if filter_type == "water":
        return {
            "animal_type": "Dog",
            "breed": {"$in": [
                "Labrador Retriever Mix", 
                "Chesapeake Bay Retriever", 
                "Chesa Bay Retr",
                "Newfoundland"
            ]},
            "sex_upon_outcome": {"$regex": r"^Intact Female"},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == "mountain":
        return {
            "animal_type": "Dog",
            "breed": {"$in": [
                "German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                "Siberian Husky", "Rottweiler"
            ]},
            "sex_upon_outcome": {"$regex": r"^Intact Male"},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == "disaster":
        return {
            "animal_type": "Dog",
            "breed": {"$in": [
                "Doberman Pinscher", "German Shepherd", "Golden Retriever",
                "Bloodhound", "Rottweiler"
            ]},
            "sex_upon_outcome": {"$regex": r"^Intact Male"},
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else: 
        return {}

@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
# Code to filter interactive data table with MongoDB queries    
    query = build_query(filter_type)
    dff = pd.DataFrame.from_records(shelter.read(query))
    if dff.empty:
        return []
    if "_id" in dff.columns:
        dff.drop(columns=["_id"], inplace=True)
    
    dff = dff.replace({np.nan: None})
    return dff.to_dict("records")

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
# Pie Chart
    if not viewData:
        return html.Div("No data to display.")
    dff = pd.DataFrame(viewData)
    dff = dff.replace({np.nan: None})
    name_col = "breed" if "breed" in dff.columns else dff.columns[0]
    fig = px.pie(dff, names=name_col, title="Breed Distribution of Current Filter")
    return dcc.Graph(figure=fig)
    
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_styles(selected_columns, selected_rows):
    styles = []
    
    if selected_columns:
        styles += [{
            'if': { 'column_id': i },
            'background_color': '#D2F3FF'
    } for i in selected_columns]
    
    if selected_rows:
        styles += [{
            'if': { 'row_index': i },
            'background_color': '#D2F3FF'
    } for i in selected_rows]

    return styles


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:27702/
